In [5]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

# Load the CSV
cardioDf = pd.read_csv('../Resources/cardio_train.csv', sep=';')
cardioDf.head(20)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
5,8,21914,1,151,67.0,120,80,2,2,0,0,0,0
6,9,22113,1,157,93.0,130,80,3,1,0,0,1,0
7,12,22584,2,178,95.0,130,90,3,3,0,0,1,1
8,13,17668,1,158,71.0,110,70,1,1,0,0,1,0
9,14,19834,1,164,68.0,110,60,1,1,0,0,0,0


In [6]:
# Scale data 
scaler = StandardScaler()
cardioAttributes = cardioDf.drop('cardio', axis=1)
cardioLabels = cardioDf['cardio']
cardioAttributesScaled = scaler.fit_transform(cardioAttributes)

In [7]:
cardioAttributesScaled

array([[-1.73208045, -0.43606151,  1.36405487, ..., -0.31087913,
        -0.23838436,  0.49416711],
       [-1.73204579,  0.30768633, -0.73310834, ..., -0.31087913,
        -0.23838436,  0.49416711],
       [-1.73201113, -0.24799666, -0.73310834, ..., -0.31087913,
        -0.23838436, -2.02360695],
       ...,
       [ 1.7338537 , -0.16328642,  1.36405487, ..., -0.31087913,
         4.19490608, -2.02360695],
       [ 1.73392303,  1.20058905, -0.73310834, ..., -0.31087913,
        -0.23838436, -2.02360695],
       [ 1.73395769,  0.43414373, -0.73310834, ..., -0.31087913,
        -0.23838436,  0.49416711]])

In [ ]:
#PCA Placeholder Code for future

In [9]:
# Split training/test datasets
trainingCardioAttributes, testingCardioAttributes, trainingCardioLabels, testingCardioLabels = train_test_split(cardioAttributesScaled, cardioLabels, random_state=78)


In [10]:
# Create a random forest classifier.
rfModel = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rfModel = rfModel.fit(trainingCardioAttributes, trainingCardioLabels)

# Evaluate the model
cardioLabelPredictions = rfModel.predict(testingCardioAttributes)

In [14]:
print(f"Accuracy Score: {accuracy_score(testingCardioLabels, cardioLabelPredictions)}")

Accuracy Score: 0.7272
